In [1]:
import pandas as pd
from sqlalchemy import create_engine
import json
from pandas.io.json import json_normalize
import requests
from amadeus import Location
from amadeus import Client, ResponseError

In [2]:
amadeus = Client(
    client_id='mfWDFAuJHzCAyiuyrF2ZNPLt86x6pBHe',
    client_secret='20kJCr29aZVItX2F'
)

try:
    response = amadeus.reference_data.urls.checkin_links.get(airlineCode='BA')
    print(response.data)
except ResponseError as error:
    print(error)

[{'type': 'checkin-link', 'id': 'BAEN-GBAll', 'href': 'https://www.britishairways.com/travel/olcilandingpageauthreq/public/en_gb', 'channel': 'All'}, {'type': 'checkin-link', 'id': 'BAEN-GBMobile', 'href': 'https://www.britishairways.com/travel/managebooking/public/en_gb/device-mobile?bookingRef={PNR}&lastname={LAST}', 'channel': 'Mobile', 'parameters': {'LAST': {'description': 'Passenger Last Name', 'type': 'string'}, 'PNR': {'description': 'Record Locator', 'type': 'string', 'format': '([a-zA-Z]|[0-9]){6}'}}}, {'type': 'checkin-link', 'id': 'BAEN-GBWeb', 'href': 'https://www.britishairways.com/travel/managebooking/public/en_gb?&bookingRef={PNR}&lastname={LAST}', 'channel': 'Web', 'parameters': {'LAST': {'description': 'Passenger Last Name', 'type': 'string'}, 'PNR': {'description': 'Record Locator', 'type': 'string', 'format': '([a-zA-Z]|[0-9]){6}'}}}]


## Where did most people fly to from SFO in Valentineds day 2018? (from Feb 01 to Feb 28 - full month)

In [3]:
response_best_place = amadeus.travel.analytics.air_traffic.traveled.get(originCityCode='SFO', period='2018-02')
response_best_place = response_best_place.data
print(response_best_place)

[{'type': 'air-traffic', 'destination': 'LAS', 'subType': 'TRAVELED', 'analytics': {'flights': {'score': 63}, 'travelers': {'score': 100}}}, {'type': 'air-traffic', 'destination': 'LAX', 'subType': 'TRAVELED', 'analytics': {'flights': {'score': 86}, 'travelers': {'score': 93}}}, {'type': 'air-traffic', 'destination': 'NYC', 'subType': 'TRAVELED', 'analytics': {'flights': {'score': 100}, 'travelers': {'score': 91}}}, {'type': 'air-traffic', 'destination': 'CHI', 'subType': 'TRAVELED', 'analytics': {'flights': {'score': 52}, 'travelers': {'score': 36}}}, {'type': 'air-traffic', 'destination': 'BLR', 'subType': 'TRAVELED', 'analytics': {'flights': {'score': 17}, 'travelers': {'score': 13}}}, {'type': 'air-traffic', 'destination': 'MAD', 'subType': 'TRAVELED', 'analytics': {'flights': {'score': 27}, 'travelers': {'score': 11}}}, {'type': 'air-traffic', 'destination': 'SIN', 'subType': 'TRAVELED', 'analytics': {'flights': {'score': 14}, 'travelers': {'score': 10}}}, {'type': 'air-traffic', 

In [4]:
# print(json.dumps(response, indent=4, sort_keys=True))
print(json.dumps(response_best_place, indent=4, sort_keys=True))

[
    {
        "analytics": {
            "flights": {
                "score": 63
            },
            "travelers": {
                "score": 100
            }
        },
        "destination": "LAS",
        "subType": "TRAVELED",
        "type": "air-traffic"
    },
    {
        "analytics": {
            "flights": {
                "score": 86
            },
            "travelers": {
                "score": 93
            }
        },
        "destination": "LAX",
        "subType": "TRAVELED",
        "type": "air-traffic"
    },
    {
        "analytics": {
            "flights": {
                "score": 100
            },
            "travelers": {
                "score": 91
            }
        },
        "destination": "NYC",
        "subType": "TRAVELED",
        "type": "air-traffic"
    },
    {
        "analytics": {
            "flights": {
                "score": 52
            },
            "travelers": {
                "score": 36
            }
     

# Store JSON data into a DataFrame #1

In [5]:
#json_file = "../Resources/customer_location.json"
best_places_df = pd.DataFrame.from_dict(json_normalize(response_best_place), orient='columns')
best_places_df

,analytics.flights.score,analytics.travelers.score,destination,subType,type
0,63,100,LAS,TRAVELED,air-traffic
1,86,93,LAX,TRAVELED,air-traffic
2,100,91,NYC,TRAVELED,air-traffic
3,52,36,CHI,TRAVELED,air-traffic
4,17,13,BLR,TRAVELED,air-traffic
5,27,11,MAD,TRAVELED,air-traffic
6,14,10,SIN,TRAVELED,air-traffic
7,21,7,BCN,TRAVELED,air-traffic
8,11,7,SNA,TRAVELED,air-traffic
9,17,5,IST,TRAVELED,air-traffic


# Clean DataFrame - #1 

## TRANSFORM: ADDING REGION OF AIRPORT TO THE DATA FRAME

In [6]:
# Get a specific city or airport based on its id

airport_codes = best_places_df['destination'].tolist()
# airport_codes

test_list = []

for airport in airport_codes:
    try:
        response = amadeus.reference_data.locations.get(
        keyword=airport,
        subType=Location.ANY)

     #  print(airport) 

        test_list.append(response.data[0]['address']['regionCode'])

    except:
        test_list.append('not here')
    

best_places_df['country'] = pd.DataFrame(test_list)

In [7]:
best_places_df

,analytics.flights.score,analytics.travelers.score,destination,subType,type,country
0,63,100,LAS,TRAVELED,air-traffic,NAMER
1,86,93,LAX,TRAVELED,air-traffic,NAMER
2,100,91,NYC,TRAVELED,air-traffic,NAMER
3,52,36,CHI,TRAVELED,air-traffic,NAMER
4,17,13,BLR,TRAVELED,air-traffic,ASIA
5,27,11,MAD,TRAVELED,air-traffic,EUROP
6,14,10,SIN,TRAVELED,air-traffic,ASIA
7,21,7,BCN,TRAVELED,air-traffic,EUROP
8,11,7,SNA,TRAVELED,air-traffic,NAMER
9,17,5,IST,TRAVELED,air-traffic,not here


In [8]:
# print(json.dumps(response.data, indent=4, sort_keys=True))
# print(response.data[0]['address']['countryName'])

# TRANSFORM: GET RID OF ROWS WE DONT NEED FROM THE JSON CALL

In [9]:
best_places_df_new = best_places_df[best_places_df.country != 'NAMER']
best_places_df_new = best_places_df_new[best_places_df.country != 'not here']
best_places_df_new

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,analytics.flights.score,analytics.travelers.score,destination,subType,type,country
4,17,13,BLR,TRAVELED,air-traffic,ASIA
5,27,11,MAD,TRAVELED,air-traffic,EUROP
6,14,10,SIN,TRAVELED,air-traffic,ASIA
7,21,7,BCN,TRAVELED,air-traffic,EUROP


# Take the top 3 destination airports and we query the price of the flights for same date range in 2019


In [10]:
#testing the api call
low_price = amadeus.shopping.flight_offers.get(origin='SFO', destination='JFK', departureDate='2020-02-14')
low_price.data

[{'type': 'flight-offer',
  'id': '1565318001456-1440499534',
  'offerItems': [{'services': [{'segments': [{'flightSegment': {'departure': {'iataCode': 'SFO',
          'terminal': '1',
          'at': '2020-02-14T09:26:00-08:00'},
         'arrival': {'iataCode': 'LAX',
          'terminal': '2',
          'at': '2020-02-14T11:05:00-08:00'},
         'carrierCode': 'DL',
         'number': '2313',
         'aircraft': {'code': '319'},
         'operating': {'carrierCode': 'DL', 'number': '2313'},
         'duration': '0DT1H39M'},
        'pricingDetailPerAdult': {'travelClass': 'ECONOMY',
         'fareClass': 'K',
         'availability': 9,
         'fareBasis': 'KA3UA0MA'}},
       {'flightSegment': {'departure': {'iataCode': 'LAX',
          'terminal': '2',
          'at': '2020-02-14T22:30:00-08:00'},
         'arrival': {'iataCode': 'JFK',
          'terminal': '4',
          'at': '2020-02-15T06:59:00-05:00'},
         'carrierCode': 'DL',
         'number': '1908',
         '

In [11]:
# low_price_df = pd.DataFrame.from_dict(json_normalize(low_price.data), orient='columns')
# low_price_df

# Defining the json objects we will append to the table. Price, Departure_date and Arrival_date
    

In [12]:
# price == low_price.data[0]['offerItems'][0]['price']['total']
# taxes ==low_price.data[0]['offerItems'][0]['price']['totalTaxes']
# departure_date == low_price.data[0]['offerItems'][0]['services'][0]['segments'][0]['flightSegment']['departure']['at']
# arrival_date == low_price.data[0]['offerItems'][0]['services'][0]['segments'][0]['flightSegment']['arrival']['at']

In [13]:
# Creating a list of final airport codes that we will loop through for API call
destinations = best_places_df_new['destination'].tolist()
destinations

['BLR', 'MAD', 'SIN', 'BCN']

In [14]:
# low_price_list = []
# low_price_df = {}

# for destination in destinations:
    
#     response = amadeus.shopping.flight_offers.get(origin='SFO', destination=destination, departureDate='2020-02-14')
#     price_list = []
#     departure_list = []
#     arrival_list = []
    
#     for i in range(len(low_price.data)):
#         try:
#             low_price_list.append(destination)
#             low_price_df['destination'] = destination
          
#             price_list.append(response.data[i]['offerItems'][0]['price']['total'])
#             low_price_df['price'] = price_list
  
#             departure_list.append(response.data[i]['offerItems'][0]['services'][0]['segments'][0]['flightSegment']['departure']['at'])
#             low_price_df['departure_date'] = departure_list  
                    
#             arrival_list.append(response.data[i]['offerItems'][0]['services'][0]['segments'][0]['flightSegment']['arrival']['at'])
#             low_price_df['arrival_date'] = arrival_list  
            
#         except:
#             low_price_list.append('not here')

In [15]:
results_leave = []
for destination in destinations:
    
    response = amadeus.shopping.flight_offers.get(origin='SFO', destination=destination, departureDate='2020-02-14')
#     price_list = []
#     departure_list = []
#     arrival_list = []
    
    for i in range(len(low_price.data)):
        try:
            item = {'destination': destination, \
                    'price_leave': response.data[i]['offerItems'][0]['price']['total'], \
                    'price_leave_tax': response.data[i]['offerItems'][0]['price']['totalTaxes'],\
                    'departure_date': response.data[i]['offerItems'][0]['services'][0]['segments'][0]['flightSegment']['departure']['at'], \
                    'arrival_date': response.data[i]['offerItems'][0]['services'][0]['segments'][0]['flightSegment']['arrival']['at'],\
                    'carrier_code': response.data[i]['offerItems'][0]['services'][0]['segments'][0]['flightSegment']['carrierCode'],\
                    'carrier_number': response.data[i]['offerItems'][0]['services'][0]['segments'][0]['flightSegment']['number'],\
                    'direction': 'outbound'
                   }
            results_leave.append(item)
        

        except:
            pass

In [16]:
# to pandas DF
results_df = pd.DataFrame(results_leave)
results_df.head()

,arrival_date,carrier_code,carrier_number,departure_date,destination,direction,price_leave,price_leave_tax
0,2020-02-14T17:07:00-05:00,AF,2541,2020-02-14T08:30:00-08:00,BLR,outbound,1139.88,444.88
1,2020-02-14T12:30:00-08:00,EK,3225,2020-02-14T10:30:00-08:00,BLR,outbound,1093.09,341.09
2,2020-02-15T18:45:00+08:00,CX,879,2020-02-14T11:20:00-08:00,BLR,outbound,721.74,185.74
3,2020-02-14T15:10:00-05:00,AF,5983,2020-02-14T07:45:00-08:00,BLR,outbound,1139.88,444.88
4,2020-02-15T09:55:00+01:00,UA,58,2020-02-14T13:50:00-08:00,BLR,outbound,1067.01,301.01


query the low price flight to SFO (Feb 16,2020)

In [17]:
results_back = []
for destination in destinations:
    
    response = amadeus.shopping.flight_offers.get(origin=destination, destination='SFO', departureDate='2020-02-16')
    
    for i in range(len(low_price.data)):
        try:
            item = {'origin': destination, \
                    'price_back': response.data[i]['offerItems'][0]['price']['total'], \
                    'price_back_tax': response.data[i]['offerItems'][0]['price']['totalTaxes'],\
                    'departure_date': response.data[i]['offerItems'][0]['services'][0]['segments'][0]['flightSegment']['departure']['at'], \
                    'arrival_date': response.data[i]['offerItems'][0]['services'][0]['segments'][0]['flightSegment']['arrival']['at'],\
                    'carrier_code': response.data[i]['offerItems'][0]['services'][0]['segments'][0]['flightSegment']['carrierCode'],\
                    'carrier_number': response.data[i]['offerItems'][0]['services'][0]['segments'][0]['flightSegment']['number'],\
                    'direction': 'inbound'
                   }
            results_back.append(item)
        
            
        except:
            pass

In [18]:
# to pandas DF
results_df_back = pd.DataFrame(results_back)
results_df_back

,arrival_date,carrier_code,carrier_number,departure_date,direction,origin,price_back,price_back_tax
0,2020-02-16T13:00:00+04:00,EK,565,2020-02-16T10:25:00+05:30,inbound,BLR,975.55,362.55
1,2020-02-16T09:00:00+01:00,UA,8897,2020-02-16T03:35:00+05:30,inbound,BLR,2674.49,413.49
2,2020-02-16T09:00:00+01:00,LH,755,2020-02-16T03:35:00+05:30,inbound,BLR,982.59,331.59
3,2020-02-16T07:05:00+08:00,SQ,505,2020-02-16T00:05:00+05:30,inbound,BLR,3549.09,228.09
4,2020-02-16T09:00:00+01:00,AC,9057,2020-02-16T03:35:00+05:30,inbound,BLR,914.34,312.34
5,2020-02-16T22:15:00+05:30,UK,818,2020-02-16T19:30:00+05:30,inbound,BLR,909.98,307.98
6,2020-02-16T09:00:00+01:00,UA,8897,2020-02-16T03:35:00+05:30,inbound,BLR,932.54,330.54
7,2020-02-17T06:10:00+08:00,SQ,503,2020-02-16T23:10:00+05:30,inbound,BLR,3549.09,228.09
8,2020-02-16T09:00:00+01:00,UA,8897,2020-02-16T03:35:00+05:30,inbound,BLR,2674.49,413.49
9,2020-02-16T19:00:00+08:00,MI,423,2020-02-16T11:55:00+05:30,inbound,BLR,1563.54,333.54


# Connect to local database

In [25]:
rds_connection_string = "postgres:postgres@localhost:5432/vacation_planning"
engine = create_engine(f'postgresql://{rds_connection_string}')

## Check for tables

In [26]:
engine.table_names()

['flights_outbound', 'flights_inbound']

In [29]:
results_df_back.to_sql(name='flights_inbound',con=engine, if_exists='append', index = False)

In [69]:
BLR = pd.read_sql_query("select origin, carrier_code,  carrier_number, price_back, price_back_tax, \
                  price_back + price_back_tax AS total_Price, direction \
                  from flights_inbound \
                where flights_inbound.origin = 'BLR' limit 1" , con = engine)
MAD = pd.read_sql_query("select origin, carrier_code,  carrier_number, price_back, price_back_tax, \
                  price_back + price_back_tax AS total_Price,direction \
                  from flights_inbound \
                where flights_inbound.origin = 'MAD' limit 1" , con = engine)
BCN = pd.read_sql_query("select origin, carrier_code,  carrier_number, price_back, price_back_tax, \
                  price_back + price_back_tax AS total_Price,direction \
                  from flights_inbound \
                where flights_inbound.origin = 'BCN' limit 1" , con = engine)
SIN = pd.read_sql_query("select origin, carrier_code,  carrier_number, price_back, price_back_tax, \
                  price_back + price_back_tax AS total_Price,direction \
                  from flights_inbound \
                where flights_inbound.origin = 'SIN' limit 1" , con = engine)

cheap_inbound  = BLR.append(MAD)
cheap_inbound  = cheap_inbound.append(SIN)
cheap_inbound  = cheap_inbound.append(BCN)
cheap_inbound

,origin,carrier_code,carrier_number,price_back,price_back_tax,total_price,direction
0,BLR,EK,565,975.55,362.55,1338.10,inbound
0,MAD,IB,3714,2150.12,190.12,2340.24,inbound
0,SIN,AC,6303,732.74,233.74,966.48,inbound
0,BCN,SN,9035,2646.60,194.60,2841.20,inbound


In [31]:
results_df.to_sql(name='flights_outbound',con=engine, if_exists='append', index = False)

In [34]:
pd.read_sql_query('select * from flights_outbound', con = engine).head()

,id,destination,departure_date,arrival_date,price_leave,price_leave_tax,carrier_code,carrier_number,direction
0,1,BLR,2020-02-14T08:30:00-08:00,2020-02-14T17:07:00-05:00,1139.88,444.88,AF,2541,outbound
1,2,BLR,2020-02-14T10:30:00-08:00,2020-02-14T12:30:00-08:00,1093.09,341.09,EK,3225,outbound
2,3,BLR,2020-02-14T11:20:00-08:00,2020-02-15T18:45:00+08:00,721.74,185.74,CX,879,outbound
3,4,BLR,2020-02-14T07:45:00-08:00,2020-02-14T15:10:00-05:00,1139.88,444.88,AF,5983,outbound
4,5,BLR,2020-02-14T13:50:00-08:00,2020-02-15T09:55:00+01:00,1067.01,301.01,UA,58,outbound


In [72]:
BLR2 = pd.read_sql_query("select destination, carrier_code,  carrier_number, price_leave, price_leave_tax, \
                  price_back + price_back_tax AS total_Price, direction \
                  from flights_outbound \
                where flights_outbound.destination = 'BLR' limit 1" , con = engine)
MAD2 = pd.read_sql_query("select destination, carrier_code,  carrier_number, price_leave, price_leave_tax, \
                  price_back + price_back_tax AS total_Price, direction \
                  from flights_outbound \
                where flights_outbound.origin = 'MAD' limit 1" , con = engine)
BCN2 = pd.read_sql_query("select origin, carrier_code,  carrier_number, price_leave, price_leave_tax, \
                  price_back + price_back_tax AS total_Price, direction \
                  from flights_outbound \
                where flights_outbound.origin = 'BCN' limit 1", con = engine)
SIN2 = pd.read_sql_query("select origin, carrier_code,  carrier_number, price_leave, price_leave_tax, \
                  price_back + price_back_tax AS total_Price, direction \
                  from flights_outbound \
                where flights_outbound.origin = 'SIN' limit 1" , con = engine)

cheap_outbound  = BLR2.append(MAD2)
cheap_outbound  = cheap_outbound.append(SIN2)
cheap_outbound  = cheap_outbound.append(BCN2)
cheap_outbound

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "origin" does not exist
LINE 1: select origin, carrier_code,  carrier_number, price_leave, p...
               ^

[SQL: select origin, carrier_code,  carrier_number, price_leave, price_leave_tax,                   price_back + price_back_tax AS total_Price, direction                   from flights_outbound                 where flights_outbound.origin = 'BLR' limit 1]
(Background on this error at: http://sqlalche.me/e/f405)